In [1]:
import pandas as pd
import numpy as np
from data_path import data_path
dataPath = data_path.dataPath

data = pd.read_parquet(dataPath + 'train_series.parquet')
labels = pd.read_csv(dataPath + 'train_events.csv')

display(data)
display(labels)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
127946335,fe90110788d2,592375,2017-09-08T00:14:35-0400,-27.277500,0.0204
127946336,fe90110788d2,592376,2017-09-08T00:14:40-0400,-27.032499,0.0233
127946337,fe90110788d2,592377,2017-09-08T00:14:45-0400,-26.841200,0.0202
127946338,fe90110788d2,592378,2017-09-08T00:14:50-0400,-26.723900,0.0199


,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400
...,...,...,...,...,...
14503,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400
14504,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400
14505,fe90110788d2,34,wakeup,581604.0,2017-09-07T09:17:00-0400
14506,fe90110788d2,35,onset,NaN,NaN


In [2]:
sample_data = False
sample_size = 10

if sample_data:
    series_id_sample = data["series_id"].sample(sample_size)

    data_sample = data[data["series_id"].isin(series_id_sample)]
    label_sample = labels[labels["series_id"].isin(series_id_sample)]
else:
    data_sample = data
    label_sample = labels

merged_data = data_sample.merge(label_sample, on=['series_id', "step", "timestamp"], how='left')
merged_data_only_labels = merged_data[np.logical_not(merged_data["event"].isna())]


In [3]:
unique_people = list(set(data["series_id"].values))

In [4]:
def getTimeFromTimestamp(timestamp):
    sum = 0
    sum += int(timestamp[:2]) * 3600
    sum += int(timestamp[3:5]) * 60
    sum += int(timestamp[6:8])
    return sum

In [5]:
series_length = int(getTimeFromTimestamp("24:00:00") / 5)

def fix_daylight_savings(data):
    times = data["timestamp"].apply(lambda x: getTimeFromTimestamp(x[11:-5]))

    for i in range(len(times)):
        if (5 * i) % (series_length * 5) != times[i]:
            index = i
            time_stamp_at_day_light_savings = data["timestamp"].values[i]
            break

    #index if the index at day light savings and time_stamp_at_day_light_savings is the time stamp at day light savings
    #if time_stamp_at_day_light_savings is 01:00:00 then we gained an hour and need to remove the hour from the data
    #if time_stamp_at_day_light_savings is 03:00:00 then we lost an hour and need to add an hour to the data which will be the same as the previous hour
        
    try:
        data.drop("level_0", axis=1, inplace=True)
    except:
        pass

    if time_stamp_at_day_light_savings[11:-5] == "01:00:00":
        drop_range = range(index - 720, index)
        data = data.drop(drop_range).reset_index()
    elif time_stamp_at_day_light_savings[11:-5] == "03:00:00":
        insert_hour = data[index - 720: index]
        concat_list = [data[:index], insert_hour, data[index:]]
        data = pd.concat(concat_list).reset_index()

    return data
                

In [6]:
import torch

def make_label_tensor(data, labels):
    #making the label for any step 0 if the person is awake and 1 if the person is asleep
    label_tensor_person = torch.zeros((len(data)))
    first_step = data["step"].values[0]
    first_event = labels["event"].values[0]


    #num refers to the coorect label to start with
    #if the first event is onset then the first label should be 1(awake) and if the first event is wakeup then the first label should be 0(asleep)
    if (first_event == "onset"):
        num = 1
    elif (first_event == "wakeup"):
        num = 0

    end_index = 0
    for j in range(int(len(labels) - 1)):
        start_index = labels["step"].values[j] - first_step
        end_index = labels["step"].values[j + 1] - first_step

        label_tensor_person[start_index:end_index] = num
        #alternates num between 0 and 1
        num = (num + 1) % 2

    #this part takes care of everything after the last event
    label_tensor_person[end_index:] = num

    return label_tensor_person

In [7]:
import torch
import numpy as np

series_length = int(getTimeFromTimestamp("24:00:00") / 5)
data_list = []
labels_list = []


for i in range(len(unique_people)):
    person_data = merged_data[merged_data["series_id"] == unique_people[i]].reset_index()
    person_labels = merged_data_only_labels[merged_data_only_labels["series_id"] == unique_people[i]].reset_index()

    #merge removes some peoples labels for some reason so we drop these people
    if len(person_labels) == 0:
        continue

    #trimming the ends of each persons data and getting rid of labels that are no longer in the data
    startTime = getTimeFromTimestamp(person_data["timestamp"].values[0][11:-5])
    endTime = getTimeFromTimestamp(person_data["timestamp"].values[-1][11:-5])

    person_data = person_data[series_length - int(startTime / 5): -1 * int(endTime / 5 + 1)].reset_index()
    person_labels = person_labels[person_labels["step"].isin(person_data["step"])].reset_index()

    #fixing daylight savings so that the length of the data is made of whole days
    if (len(person_data) % 17280) == 720 or (len(person_data) % 17280) == 16560:
        person_data = fix_daylight_savings(person_data)


    label_tensor_person = make_label_tensor(person_data, person_labels)

    
    data_list.append(person_data)
    labels_list.append(label_tensor_person)
     

In [8]:
#concatenating all the data and labels together and reshaping them into shape (batch_size, series_length, num_features(2 for data and 1 for label))
trimmed_data = pd.concat(data_list)

enmo_list = trimmed_data["enmo"].values
anglez_list = trimmed_data["anglez"].values
enmo_tensor = torch.tensor(enmo_list)
anglez_tensor = torch.tensor(anglez_list)

data_tensor = torch.stack((enmo_tensor, anglez_tensor), dim=1)
label_tensor = torch.cat(labels_list)

data_tensor = data_tensor.reshape((len(trimmed_data) // series_length, series_length, 2))
label_tensor = label_tensor.reshape((len(trimmed_data) // series_length, series_length))

In [9]:
torch.save(data_tensor, "batched_data\\features")
torch.save(label_tensor, "batched_data\\labels")